In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas

In [96]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [114]:
# # for 3 second vggish
pod_train_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/vggish_3sec/vggish_3sec_pod_train_embeddings.npy")
#[:852, :, :]
ad_train_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/vggish_3sec/vggish_3sec_ad_train_embeddings.npy")

#live_radio_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/Vggish_live_pod5_3sec/Vggish_live_pod5_3_sec_embeddings_new.npy")

#live_radio_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/Vggish_live_pod2_3sec/Vggish_live_pod2_3_sec_embeddings_new.npy")[:826, :, :]

#live_radio_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/Vggish_live_pod1_3sec/Vggish_live_pod1_3_sec_embeddings_new.npy")

# cnn radio live
live_radio_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/Vggish_live_radio_3_sec/Vggish_live_radio_3_sec_embeddings_new.npy")

#pod_val_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/vggish_3sec/vggish_3sec_pod_val_embeddings.npy")
#ad_val_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/vggish_3sec/vggish_3sec_ad_val_embeddings.npy")

#pod_test_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/vggish_3sec/vggish_3sec_pod_test_embeddings.npy")
#ad_test_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/vggish_3sec/vggish_3sec_ad_test_embeddings.npy")


In [115]:
print("pod train embeddings shape:", pod_train_embeddings.shape)
print("ad train embeddings shape:", ad_train_embeddings.shape)
print("live radio embeddings shape:", live_radio_embeddings.shape)

pod train embeddings shape: (5260, 3, 128)
ad train embeddings shape: (852, 3, 128)
live radio embeddings shape: (709, 3, 128)


In [116]:
def convert_to_2d_array(three_d_array):
    # Get the dimensions of the input array
    depth, rows, cols = three_d_array.shape

    # Reshape each 2D array to 1D and concatenate them
    flattened_arrays = [matrix.flatten() for matrix in three_d_array]
    two_d_array = np.vstack(flattened_arrays)

    return two_d_array

In [117]:
pod_train_embeddings = convert_to_2d_array(pod_train_embeddings)
ad_train_embeddings = convert_to_2d_array(ad_train_embeddings)

live_radio_embeddings = convert_to_2d_array(live_radio_embeddings)

In [118]:
print("pod train embeddings shape:", pod_train_embeddings.shape)
print("ad train embeddings shape:", ad_train_embeddings.shape)
print("live radio embeddings shape:", live_radio_embeddings.shape)

pod train embeddings shape: (5260, 384)
ad train embeddings shape: (852, 384)
live radio embeddings shape: (709, 384)


In [119]:
train_embeddings = np.concatenate((pod_train_embeddings, ad_train_embeddings))

In [120]:
def concatenate_zeros_and_ones(podcast_length, commercials_length):
    # Create array of zeros with size of podcast array
    zeros_array = np.zeros(podcast_length, dtype=int)

    # Create array of ones with size of commercials array
    ones_array = np.ones(commercials_length, dtype=int)

    # Concatenate arrays
    concatenated_array = np.concatenate((zeros_array, ones_array))

    return concatenated_array


podcast_length = pod_train_embeddings.shape[0]
ads_length = ad_train_embeddings.shape[0]

In [121]:
train_labels = concatenate_zeros_and_ones(pod_train_embeddings.shape[0], ad_train_embeddings.shape[0])
print(train_labels.shape)

(6112,)


In [122]:
#live_radio_excel_path = "/content/drive/MyDrive/AD-Blocker Project/live_pod5/audio_segments.xlsx"


#live_radio_excel_path = "/content/drive/MyDrive/AD-Blocker Project/live_pod2/audio_segments.xlsx"

#live_radio_excel_path = "/content/drive/MyDrive/AD-Blocker Project/live_pod1/audio_segments.xlsx"

# cnn live radio
live_radio_excel_path = "/content/drive/MyDrive/AD-Blocker Project/real_time_audio/audio_segments.xlsx"

df = pd.read_excel(live_radio_excel_path)
labels = df['Label'].to_numpy()

# Step 6: Take all elements except for the last item
labels_except_last = labels[:-1]
#labels_except_last = labels[:826]
labels_except_last = labels_except_last.astype(int)
# Display the array
print(labels_except_last)

print(labels_except_last.shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [123]:
# Define sample weights
# Assuming you have a binary classification problem with labels 0 (podcast) and 1 (commercial)
# Assign weight 1 to podcast samples and weight 6 to commercial samples
sample_weights = {0: 1, 1: 1}

def train_svm_model(embeddings, labels, weights=None):
    """
    Train an SVM model using the provided embeddings and labels.

    Parameters:
    - embeddings: List of embeddings (list of arrays).
    - labels: List of corresponding labels.
    - weights: Dictionary specifying the class weights. Default is None.

    Returns:
    - model: Trained SVM model.
    """
    # Initialize SVM model with linear kernel and class weights
    model = SVC(kernel='linear', class_weight=weights)
    # Train the model
    model.fit(embeddings, labels)
    return model

In [125]:
# Train the SVM model with sample weights
model = train_svm_model(train_embeddings, train_labels)

In [126]:
import numpy as np
from sklearn.metrics import accuracy_score

def test_model(model, test_embeddings, test_labels):
    """
    Test the trained model on a separate test dataset.

    Parameters:
    - model: Trained machine learning model.
    - test_embeddings: Embeddings of the test dataset.
    - test_labels: Labels of the test dataset.

    Returns:
    - test_accuracy: Accuracy of the model on the test dataset.
    - predicted_labels: Predicted labels for the test dataset.
    - actual_labels: Actual labels of the test dataset.
    """
    # Make predictions on the test dataset
    predicted_labels = model.predict(test_embeddings)

    # Calculate test accuracy
    test_accuracy = accuracy_score(test_labels, predicted_labels)

    # Print test accuracy
    print("Test Accuracy:", test_accuracy)

    # Print predicted and actual labels for inspection
    #print("Predicted labels:")
    #print(predicted_labels)
    #print("Actual labels:")
    #print(test_labels)

    # Find and print indices where predicted and actual labels differ
    differing_indices = np.where(predicted_labels != test_labels)[0]
    #print("Indices with differences between predicted and actual labels:")
    #print(differing_indices)

    # Return test accuracy and predicted/actual labels
    return test_accuracy, predicted_labels, test_labels, differing_indices

In [127]:
# Call the function by passing the required arguments
test_accuracy, test_predicted_labels, test_actual_labels, differing_indices = test_model(model, live_radio_embeddings, labels_except_last)

# Now you can use the returned values as needed
print("Test Accuracy:", test_accuracy)

# Example array
large_array = differing_indices
# Set print options to display the full array
np.set_printoptions(threshold=np.inf)

# Print the array
print(large_array + 1)

Test Accuracy: 0.7390691114245416
Test Accuracy: 0.7390691114245416
[ 73  78  81  82  83  84  86  87  88  89  96  98  99 100 102 103 104 105
 109 110 112 113 114 118 120 121 122 123 125 129 130 133 135 136 137 138
 139 140 141 143 144 145 146 149 151 152 153 155 156 157 158 159 160 161
 162 164 165 167 169 170 171 172 173 174 175 177 178 179 181 310 311 316
 319 320 321 323 324 329 332 336 337 339 340 341 342 345 346 347 349 351
 353 354 355 358 359 361 362 363 364 365 366 367 368 371 372 379 380 381
 386 388 389 390 394 396 398 400 402 403 404 405 406 407 408 409 410 411
 412 413 415 416 417 418 419 421 422 423 424 425 426 427 428 429 430 431
 432 433 434 435 437 441 444 446 447 448 449 450 451 452 460 464 465 469
 470 476 477 478 486 658 674 677 678 680 681 686 691 692 695 697 698 700
 702 703 705 706 709]


In [128]:
def calculate_confusion_matrix(actual_labels, predicted_labels):
    """
    Calculate the confusion matrix based on actual and predicted labels.

    Parameters:
    actual_labels (list): List of actual labels (-1 for ads, 1 for podcasts).
    predicted_labels (list): List of predicted labels (-1 for ads, 1 for podcasts).

    Returns:
    tuple: A tuple containing true positives, false positives, true negatives, and false negatives.
    """
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    for actual, predicted in zip(actual_labels, predicted_labels):
        if actual == 0 and predicted == 0:  # true_negatives
            true_negatives += 1
        elif actual == 1 and predicted == 0:  # false_negatives
            false_negatives += 1
        elif actual == 1 and predicted == 1:  # true_positives
            true_positives += 1
        elif actual == 0 and predicted == 1:  # false_positives
            false_positives += 1

    return true_positives, false_positives, true_negatives, false_negatives

In [129]:
# Call the function by passing the actual labels and predicted labels
true_positives, false_positives, true_negatives, false_negatives = calculate_confusion_matrix(labels_except_last, test_predicted_labels)

# Print the results
print("True Positives:", true_positives)
print("False Positives:", false_positives)
print("True Negatives:", true_negatives)
print("False Negatives:", false_negatives)

# Calculate evaluation values
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print the evaluation values
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity)
print("F1 Score:", f1_score)

True Positives: 86
False Positives: 45
True Negatives: 438
False Negatives: 140
Precision: 0.6564885496183206
Recall: 0.3805309734513274
Specificity: 0.906832298136646
F1 Score: 0.48179271708683474
